In [1]:
import pandas as pd
import numpy as np
import openpyxl

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### Load Data

In [2]:
global_cs_2023 = pd.read_excel("../Data/Global Case Sales 2023.xlsx") # (572,887  ,  20)
master_facility = pd.read_excel("../Data/Master Facility List.xlsx") # (7,126  ,  92)
assurance_volume = pd.read_csv("../Data/QSET_ASSURANCE_PROD_VOL_EXPORT.csv") # (29,177  ,  5)

water_wwd = pd.read_excel("../Data/Water & WWD Comments.xlsx")
water_wwd = water_wwd[water_wwd['Indicator'] == 'Total Wastewater Discharged (kL) [kL]']  # (9,254  ,  9)

In [3]:
column_names = [
    "Indicator_Name", "Code", "Entity_Name", 
    "Facility_ID", "Reporting_Period", "Answer"
]

numeric_input_ind = pd.read_csv("../Data/Monthly Numeric Indicator.csv", encoding='utf-16', sep='\t', header=0, names=column_names) # (197,418  ,  6)

In [4]:
column_names = [
    "Reporting_Period", "Entity_Name", "Code", "Indicator_Name", "Answer", "Unit", 
    "Help_Text", "Comments", "Frequency", "FolderPath", "Ord"
]
text_input_ind = pd.read_csv("../Data/Monthly Text Input Indicator.csv", encoding='ISO-8859-1',header=0, names=column_names) # (70,917  ,  11)

### Task 1: Text Input
- Check to see if answers for the same indicator switches one month to next, if yes outlier
- If answer is missing, outlier

- Columns to keep: Facility_ID, Facility_Name, Bottler, OU, Reporting_Period, Detection_Results

In [5]:
# Extract Facility_ID by splitting FolderPath at '>', taking the last part, and trimming whitespace.
text_input_ind['Facility_ID'] = text_input_ind['FolderPath'].apply(lambda x: x.split('>')[-1].strip())

# Convert Facility_ID to a numeric type, coercing errors to NaN, and then change to int64.
text_input_ind['Facility_ID'] = pd.to_numeric(text_input_ind['Facility_ID'], errors='coerce').astype('int64')

In [6]:
# Count the number of missing values in the "Answer" column
missing_count_answer = text_input_ind["Answer"].isnull().sum()

# Print the result
print(f"Number of missing values in 'Answer': {missing_count_answer}")

Number of missing values in 'Answer': 26106


In [7]:
# Perform a merge based on Facility_ID
text_input_ind = text_input_ind.merge(
    master_facility[['FACILITY_ID', 'FACILITY_NAME', 'BTLR_NAME_ISSCOM', 'BUNAME_BU']],
    left_on='Facility_ID',
    right_on='FACILITY_ID',
    how='left'
)

# # Drop the extra FACILITY_ID column after merge to avoid duplication
# text_input_ind = text_input_ind.drop(columns=['FACILITY_ID_x'])

# # Rename columns to match the desired names
# text_input_ind = text_input_ind.rename(columns={'FACILITY_ID_y': 'Facility_ID', 'FACILITY_NAME': 'Facility_Name'})


In [8]:
text_input_ind.shape

(70917, 16)

In [9]:
# Count the number of missing values in the "Answer" column
missing_count_answer = text_input_ind["Answer"].isnull().sum()

# Print the result
print(f"Number of missing values in 'Answer': {missing_count_answer}")

Number of missing values in 'Answer': 26106


In [10]:
# Keep only the relevant columns
cols_to_keep = ["BUNAME_BU", "Reporting_Period", "Facility_ID", "FACILITY_NAME", "BTLR_NAME_ISSCOM", "Indicator_Name", "Answer"]
text_input_ind = text_input_ind[cols_to_keep]

In [11]:
# Count the number of missing values in the "Answer" column
missing_count_answer = text_input_ind["Answer"].isnull().sum()

# Print the result
print(f"Number of missing values in 'Answer': {missing_count_answer}")

Number of missing values in 'Answer': 26106


In [12]:
text_input_ind.shape

(70917, 7)

In [13]:
text_input_ind.head()

,BUNAME_BU,Reporting_Period,Facility_ID,FACILITY_NAME,BTLR_NAME_ISSCOM,Indicator_Name,Answer
0,NA BU,1/1/2024,10011,REYES COCA-COLA BOTTLING RTM NON PRODUCTION,REYES COCA-COLA BOTTLING,Fleet Vehicle Classification,Owned and Leased Distribution
1,INSWA BU,1/1/2024,10013,HCCB SANAND,HINDUSTAN CC BVG,Fleet Vehicle Classification,Third Party Distribution Vehicles|Owned and Le...
2,NA BU,1/1/2024,10031,TCCC RTM NON PRODUCTION,NaN,Fleet Vehicle Classification,Not Applicable
3,LATAM BU,1/1/2024,10053,DUQUE DE CAXIAS,ANDINA,Fleet Vehicle Classification,Owned and Leased Distribution|Owned and Leased...
4,LATAM BU,1/1/2024,10096,UBERLANDIA CD,UBERLANDIA,Fleet Vehicle Classification,Not Applicable


In [14]:
def detect_changes(df):
    # Ensure 'Reporting_Period' is a datetime object for sorting
    df['Reporting_Period'] = pd.to_datetime(df['Reporting_Period'], format='%m/%d/%Y')

    # Sort the DataFrame by 'Facility_ID', 'Indicator_Name', and 'Reporting_Period'
    df.sort_values(by=['Facility_ID', 'Indicator_Name', 'Reporting_Period'], inplace=True)

    # Initialize 'Flag' column with default value 0
    df['Flag'] = 0
    
    # Define a function to flag changes and missing answers
    def flag_changes_with_initial_check(group):
        shifted = group['Answer'].shift()
        # Flag changes or missing answers
        flags = (group['Answer'] != shifted) | group['Answer'].isna()
        # Explicitly set the first entry to True if it is NaN
        if group['Answer'].isna().iloc[0]:
            flags.iloc[0] = True
        else:
            flags.iloc[0] = False
        return flags.astype(int)

    # Apply the flagging function to each group
    df['Flag'] = df.groupby(['Facility_ID', 'Indicator_Name'], as_index=False, group_keys=False).apply(flag_changes_with_initial_check)

    # Initialize new columns for structured results
    df['Detection_Result'] = pd.NA
    df['Detection_Details'] = pd.NA

    # Update detection result and details based on flags
    for name, group in df.groupby(['Facility_ID', 'Indicator_Name']):
        changes = group.index[group['Flag'] == 1]

        for idx in changes:
            facility_id = df.at[idx, 'Facility_ID']
            indicator_name = df.at[idx, 'Indicator_Name']
            reporting_period = df.at[idx, 'Reporting_Period'].strftime('%m/%d/%Y')

            # Correctly find the previous reporting period
            prev_idx = group.index.get_loc(idx) - 1
            
            if prev_idx >= 0:
                previous_reporting_period = group['Reporting_Period'].iloc[prev_idx].strftime('%m/%d/%Y')
                previous_answer = group['Answer'].iloc[prev_idx]
            else:
                previous_reporting_period = "N/A"
                previous_answer = None

            if pd.isna(df.at[idx, 'Answer']):
                df.at[idx, 'Detection_Result'] = "Missing Answer"
                df.at[idx, 'Detection_Details'] = f"Anomaly Detected On: {reporting_period}"
            else:
                if prev_idx >= 0:
                    new_answer = df.at[idx, 'Answer']
                    df.at[idx, 'Detection_Result'] = "Answer Changed"
                    df.at[idx, 'Detection_Details'] = (
                        f"Answer Changed From: {previous_answer} to {new_answer}\n"
                        f"Between: {previous_reporting_period} and {reporting_period}"
                    )

    # Create the flagged dataset with the specified columns
    flagged_df = df[df['Flag'] == 1][[
        'BUNAME_BU', 'Reporting_Period', 'Facility_ID', 'FACILITY_NAME', 
        'Indicator_Name', 'BTLR_NAME_ISSCOM', 'Detection_Result', 'Detection_Details'
    ]]

    return df, flagged_df

In [16]:
# Example usage
full_dataset, flagged_dataset = detect_changes(text_input_ind)
display(full_dataset.head())
display(flagged_dataset.head())

,BUNAME_BU,Reporting_Period,Facility_ID,FACILITY_NAME,BTLR_NAME_ISSCOM,Indicator_Name,Answer,Flag,Detection_Result,Detection_Details
27277,AFRICA BU,2024-01-01,12,BUKAVU,HEINEKEN,Do you have secondary operations on-site? (ie....,No,0,<NA>,<NA>
30217,AFRICA BU,2024-02-01,12,BUKAVU,HEINEKEN,Do you have secondary operations on-site? (ie....,No,0,<NA>,<NA>
33145,AFRICA BU,2024-03-01,12,BUKAVU,HEINEKEN,Do you have secondary operations on-site? (ie....,No,0,<NA>,<NA>
36064,AFRICA BU,2024-04-01,12,BUKAVU,HEINEKEN,Do you have secondary operations on-site? (ie....,No,0,<NA>,<NA>
38992,AFRICA BU,2024-05-01,12,BUKAVU,HEINEKEN,Do you have secondary operations on-site? (ie....,No,0,<NA>,<NA>


,BUNAME_BU,Reporting_Period,Facility_ID,FACILITY_NAME,Indicator_Name,BTLR_NAME_ISSCOM,Detection_Result,Detection_Details
27279,AFRICA BU,2024-01-01,12,BUKAVU,Electricity is paid by TCCS?,HEINEKEN,Missing Answer,Anomaly Detected On: 01/01/2024
30219,AFRICA BU,2024-02-01,12,BUKAVU,Electricity is paid by TCCS?,HEINEKEN,Missing Answer,Anomaly Detected On: 02/01/2024
33147,AFRICA BU,2024-03-01,12,BUKAVU,Electricity is paid by TCCS?,HEINEKEN,Missing Answer,Anomaly Detected On: 03/01/2024
36066,AFRICA BU,2024-04-01,12,BUKAVU,Electricity is paid by TCCS?,HEINEKEN,Missing Answer,Anomaly Detected On: 04/01/2024
38994,AFRICA BU,2024-05-01,12,BUKAVU,Electricity is paid by TCCS?,HEINEKEN,Missing Answer,Anomaly Detected On: 05/01/2024


In [22]:
full_dataset["BUNAME_BU"].value_counts()

LATAM BU     22762
NA BU         9854
AFRICA BU     8130
EUROPE BU     7809
GCM BU        5024
INSWA BU      4965
ASP BU        4777
EME BU        4286
JSK BU        3252
Name: BUNAME_BU, dtype: int64

In [17]:
flagged_dataset.shape

(26637, 8)

In [18]:
full_dataset.shape

(70917, 10)

### Save to CSV

In [19]:
flagged_dataset.to_csv('../Data/Output/task1.csv', index=False) 

In [20]:
full_dataset.to_csv('../Data/Output/task1_full.csv', index=False) 